In [175]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyvi import ViTokenizer, ViPosTagger
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim import similarities 
from collections import namedtuple
import gensim.utils
import re
import string
df = pd.read_csv("vicare_symptom.csv",encoding='utf-8',sep="\t")
df["Symptom_parse"] = df.Symptom.apply(lambda x: ' '.join(ViPosTagger.postagging(ViTokenizer.tokenize(x))[0]))
data = np.array(df.Symptom_parse)
titles=np.array(df.Disease)

MedicalDocument = namedtuple('MedicalDocument', 'words tags title original_number')
n=0
alldocs = []  # Will hold all docs in original order

regex = re.compile('[%s]' % re.escape(string.punctuation.replace("_",""))) #to remove punctuation
with open("../../../vietnamese-stopwords-dash.txt",encoding='utf-8') as f:
    stopwords = f.read().splitlines()
for line_no, line in enumerate(data):
    if (type(line)==str):
        if (len(line)>10):
                line = regex.sub('', line)
                tokens = gensim.utils.to_unicode(line).lower().split()
                words = tokens[0:]
                words = [word for word in words if word not in stopwords]
                tags = [n]
                title = titles[line_no]
                alldocs.append(MedicalDocument(words, tags, title, line_no))
                n=n+1


dct = Dictionary(doc.words for doc in alldocs)  # fit dictionary
corpus = [dct.doc2bow(line.words) for line in alldocs]  # convert dataset to BoW format
model_tfidf = TfidfModel(corpus)  # fit model

def search():
    doc = ' '.join(ViPosTagger.postagging(ViTokenizer.tokenize(input()))[0])
    print("\n\nQuery: " + doc)
    vec_bow = dct.doc2bow(doc.lower().split())
    vec_tfidf = model_tfidf[vec_bow]
    index = similarities.SparseMatrixSimilarity(model_tfidf[corpus], num_features=len(dct.dfs))
    sims = index[vec_tfidf]
    for ind, score in sorted(list(enumerate(sims)), key=lambda x: x[1],reverse=True)[:10]:
        print(alldocs[ind].title + '(' + str(score) + ')' + ": " + data[alldocs[ind].original_number] )

In [178]:
search()



Query: da bị mẩn đỏ và đau khớp chân
Bệnh Viêm cơ - dây chằng(0.7419773): Các triệu_chứng bao_gồm đau khớp , sưng khớp , mẩn đỏ hoặc sốt .
Bệnh Nhiễm trùng khớp gối(0.6239134): Sưng khớp , đau khớp dữ_dội , sốt , nổi mẩn đỏ , không_thể di_chuyển chân bị ảnh_hưởng .
Bệnh Nhiễm trùng khớp háng(0.6066698): Sưng khớp , đau khớp dữ_dội , sốt , nổi mẩn đỏ , không_thể di_chuyển chân bị ảnh_hưởng , nôn_mửa .
Bệnh Giả gút(0.57113105): Sưng đau khớp , mẩn đỏ xung_quanh khớp , nóng khớp , đau khớp khi di_chuyển , triệu_chứng hay tái_phát .
Bệnh Vết thương thủng ở da(0.5204871): Đau , chảy_máu , tê , yếu , mẩn đỏ , nóng da .
Bệnh Vết thương thủng ở tai(0.5204871): Đau , chảy_máu , tê , yếu , mẩn đỏ , nóng da .
Bệnh Vết thương thủng ở cánh tay(0.5204871): Đau , chảy_máu , tê , yếu , mẩn đỏ , nóng da .
Bệnh Vết thương thủng ở ngón tay(0.5204871): Đau , chảy_máu , tê , yếu , mẩn đỏ , nóng da .
Bệnh Vết thương thủng ở bàn tay(0.5204871): Đau , chảy_máu , tê , yếu , mẩn đỏ , nóng da .
Bệnh Vết thương

In [179]:
search()



Query: đau_đầu chóng_mặt buồn_nôn
Bệnh Thất điều tiểu não(0.5473324): Các triệu_chứng bao_gồm đi_lại khó_khăn , vụng_về , chóng_mặt , khó nói , đau_đầu và buồn_nôn .
Bệnh Xuất huyết tiểu não(0.51515794): Các triệu_chứng có_thể bao_gồm đau_đầu , nôn_mửa , mất phối_hợp , chóng_mặt , nhìn đôi , chóng_mặt , nói_lắp , khó nuốt .
Bệnh do giảm áp suất(0.50769675): Các triệu_chứng có_thể bao_gồm đau khớp , chóng_mặt , tê , yếu , đau_đầu và rối_loạn .
Bệnh Choáng não(0.47887325): Nhức đầu , buồn_nôn , nôn , chóng_mặt , lú_lẫn , yếu cơ , nhìn đôi , chóng_mặt .
Bệnh Độ cao(0.46464676): Buồn_nôn , nôn , đau_đầu , khó thở , chóng_mặt , ngủ kém , lơ_mơ nhầm_lẫn
Bệnh Nhức đầu(0.45366108): Đau_đầu . Đau cổ . Chóng_mặt . Buồn_nôn , ói mửa . Bất_thường về thần_kinh . Thay_đổi thị_giác
Bệnh Glôcôm góc đóng nguyên phát cấp(0.38520956): Đau mắt , đỏ mắt , đau_đầu , buồn_nôn , nôn_mửa , mất thị_giác .
Bệnh U não tế bào hình sao(0.351544): Nhức đầu : Thường là những cơn đau_đầu mới và tiến_triển chậm . . C

In [180]:
search()



Query: tâm_lý bất_ổn hay nghĩ tiêu_cực
Bệnh Rối loạn trầm cảm - BỊ LẶP(0.35901138): Buồn_rầu thường_xuyên , khó ngủ , ngủ quá nhiều , không_thể tập_trung , suy_nghĩ tiêu_cực không kiểm_soát được , không có cảm_giác ngon_miệng , nóng_tính , cảm_giác bất_lực , uống rượu nhiều hơn , ý_nghĩ chán_nản , buông_xuôi , suy_nghĩ tiêu_cực , có ý_định muốn chết hoặc có hành_vi tự_sát .
Bệnh Lạm dụng Amphetamine(0.17987202): Tâm_lý hưng_phấn , tăng sự tự_tin , tăng huyết_áp , giãn đồng_tử , đổ mồ_hôi , bồn_chồn , khó ngủ , lo_âu , khó_chịu , triệu_chứng tâm_thần , gây_hấn , bạo_lực , co_giật , sốt , hoang_tưởng , ảo_giác , lo_âu , trầm_cảm , tự_tử , kiệt_sức , suy_dinh_dưỡng .
Bệnh Quá liều Amphetamine(0.17096694): Tâm_lý hưng_phấn , tự_tin hơn , huyết_áp_cao , học_trò giãn_nở , đổ mồ_hôi , bồn_chồn , khó ngủ , lo_âu , khó_chịu , triệu_chứng tâm_thần , gây_hấn , bạo_lực , co_giật , sốt , hoang_tưởng , ảo_giác , lo_âu , trầm_cảm , tự_tử , kiệt_sức , suy_dinh_dưỡng .
Bệnh Sưng môi(0.0): Đau , sưng 

In [181]:
search()



Query: đau xương bả_vai và mỏi_cổ
Bệnh Ung thư di căn xương(0.60813236): Đau xương , gãy xương tự_phát , suy_nhược .
Bệnh Gãy xương đòn(0.57787263): Đau vai , biến_dạng xương cổ , đau vai khi cử_động , cánh_tay tê , yếu , bầm tím vùng xương cổ , đau xương cổ .
Bệnh Lao xương(0.5531922): Thương_tổn đầu_tiên của lao thường rất nhỏ và lớn dần , tuy đã có dấu_hiệu lâm_sàng nhưng chưa thay_đổi trên X - quang trong giai_đoạn đầu ( thường thấy rõ sau hàng tháng hoặc hàng năm ) . Khác viêm xương tủy có quá_trình kích_thích tạo xương mới , lao xương chỉ phá_hủy gây tiêu xương và xương chết . Bệnh thường gặp ở bệnh_nhân 20-40 tuổi .
Bệnh Mô bào Langerhans(0.47299597): Các triệu_chứng có_thể bao_gồm đau xương hoặc gãy xương , răng lung_lay , khó thở , ho hoặc phát_ban da . Một_số người có bị mở_rộng các tuyến trong cơ_thể hoặc gan to .
Bệnh Gãy xương(0.45710024): Có nhiều loại gãy xương với độ nặng khác nhau . Vùng gãy xương hay gặp là ở cổ_tay , vai , cổ_chân và khớp háng … Gãy nơi khớp háng n

In [189]:
search() #Ra kết quả hơi xấu. Lặp lại ở data nhiều quá



Query: sưng mẩn đỏ ở vùng chân
Bệnh Viêm cơ - dây chằng(0.5240023): Các triệu_chứng bao_gồm đau khớp , sưng khớp , mẩn đỏ hoặc sốt .
Bệnh Áp xe đầu ngón tay (Chín mé)(0.51996756): Đầu ngón tay đau , sưng , có mủ , mẩn đỏ , nóng .
Bệnh Vết thương thủng ở miệng(0.51924723): Đau , chảy_máu , tê , yếu , mẩn đỏ , nóng da , sưng mặt .
Bệnh Vết thương thủng ở mặt(0.51924723): Đau , chảy_máu , tê , yếu , mẩn đỏ , nóng da , sưng mặt .
Bệnh Vết thương thủng ở môi(0.51924723): Đau , chảy_máu , tê , yếu , mẩn đỏ , nóng da , sưng mặt .
Bệnh Vết thương thủng ở da(0.5032886): Đau , chảy_máu , tê , yếu , mẩn đỏ , nóng da .
Bệnh Vết thương thủng ở tai(0.5032886): Đau , chảy_máu , tê , yếu , mẩn đỏ , nóng da .
Bệnh Vết thương thủng ở cánh tay(0.5032886): Đau , chảy_máu , tê , yếu , mẩn đỏ , nóng da .
Bệnh Vết thương thủng ở ngón tay(0.5032886): Đau , chảy_máu , tê , yếu , mẩn đỏ , nóng da .
Bệnh Vết thương thủng ở bàn tay(0.5032886): Đau , chảy_máu , tê , yếu , mẩn đỏ , nóng da .


In [190]:
search()



Query: ngứa đầu và ra gàu
Bệnh Nổi mẩn ngứa(0.51713353): Da đỏ . . Ngứa .
Bệnh Gàu(0.36939096): Các triệu_chứng có_thể bao_gồm bong tróc , ngứa và đỏ da đầu .
Bệnh Đau đầu căng thẳng(0.33747754): Cảm_giác căng hoặc siết chặt ở các cơ vùng đầu và cổ . . Đầu đầu kiểu bị nén ép hoặc ê_ẩm ở đầu , tăng nhạy_cảm đau xung_quanh đầu . . Đau thường lan_tỏa khắp đầu . . Thường khó_chịu nhất ở phần sau đầu ( vùng chẩm ) và vùng cổ . . Đau_đầu thường nặng hơn khi kèm theo stress , mệt_mỏi , tiếng ồn , chói sáng hoặc vào cuối ngày .
Bệnh Chấy rận(0.28802133): Ngứa dữ_dội , mụn đỏ trên da
Bệnh Bán hẹp da quy đầu(0.2807507): Các triệu_chứng bao_gồm đau , sưng bao quy đầu và đầu dương_vật .
Bệnh Viêm loét miệng(0.27676445): Vết loét gây đau , ngứa bên trong miệng .
Bệnh Hăm(0.2757484): Bao_gồm đau , ngứa , đỏ da , khó_chịu và đôi_khi sốt .
Bệnh Chứng da vẽ nổi(0.26844907): Ngứa , nổi mề_đay .
Bệnh Đau đầu căn nguyên mạch máu thần kinh(0.2631742): Nhức đầu nặng ( cảm_giác đau_nhói ) , buồn_nôn , nôn 